# Relationships

In [14]:
from sqlalchemy import text

## Database connection

In [1]:
import yaml
from sqlalchemy import create_engine


with open("./config.yml", "r") as file:
	config = yaml.safe_load(file)
	config_OLTP = config["OLTP"]
	config_OLAP = config["OLAP"]


url_OLTP = (f"{config_OLTP["drivername"]}://{config_OLTP["user"]}:{config_OLTP["password"]}"
			f"@{config_OLTP["host"]}:{config_OLTP["port"]}/{config_OLTP["database_name"]}")

url_OLAP = (f"{config_OLAP["drivername"]}://{config_OLAP["user"]}:{config_OLAP["password"]}"
			f"@{config_OLAP["host"]}:{config_OLAP["port"]}/{config_OLAP["database_name"]}")

OLTP_connection = create_engine(url_OLTP)
OLAP_connection = create_engine(url_OLAP)

## Primary keys

In [37]:
dimension_tables = [
	"COURIER_DIMENSION",
	"CUSTOMER_DIMENSION",
	"OFFICE_DIMENSION",
	"SERVICE_STATUS_DIMENSION",
	"TIME_DIMENSION",
	"UPDATES_DIMENSION"
]

In [ ]:
def add_primary_key_dimension_table(dimension_table):
	with OLAP_connection.connect() as connection:
		with connection.begin():
			name_without_dimension = dimension_table.replace("_DIMENSION", "").lower()
			query = text(f"""
			ALTER TABLE "{dimension_table}"
			ADD CONSTRAINT pk_{name_without_dimension}_dimension PRIMARY KEY ({name_without_dimension}_id);
			""")
			connection.execute(query)

In [36]:
for dimension_table in dimension_tables:
	add_primary_key_dimension_table(dimension_table)

Error


In [ ]:
fact_tables = [
	"ACUMMULATING_SNAPSHOT_FACT_TABLE",
	"SERVICE_DAILY_FACT_TABLE",
	"SERVICE_HOUR_FACT_TABLE",
	"UPDATES_FACT_TABLE"
]

In [39]:
def add_primary_key_fact_table(fact_table):
	with OLAP_connection.connect() as connection:
		with connection.begin():
			name_without_fact_table = fact_table.replace("_FACT_TABLE", "").lower()
			query = text(f"""
			ALTER TABLE "{fact_table}"
			ADD CONSTRAINT pk_{name_without_fact_table}_fact_table PRIMARY KEY ({name_without_fact_table}_id);
			""")
			connection.execute(query)

In [40]:
for fact_table in fact_tables:
	add_primary_key_fact_table(fact_table)

## Foreign keys

In [22]:
def alter_table(table, query):
	with OLAP_connection.connect() as connection:
		with connection.begin():
			query = text(f"""
			ALTER TABLE "{table}"
			{query}
			""")
			connection.execute(query)

### `ACUMMULATING_SNAPSHOT_FACT_TABLE`

In [29]:
query = """
ADD FOREIGN KEY (request_time_id) REFERENCES "TIME_DIMENSION"(time_id),
ADD FOREIGN KEY (assignment_time_id) REFERENCES "TIME_DIMENSION"(time_id),
ADD FOREIGN KEY (pickup_time_id) REFERENCES "TIME_DIMENSION"(time_id),
ADD FOREIGN KEY (delivery_time_id) REFERENCES "TIME_DIMENSION"(time_id),
ADD FOREIGN KEY (closure_time_id) REFERENCES "TIME_DIMENSION"(time_id);
"""

alter_table("ACUMMULATING_SNAPSHOT_FACT_TABLE", query)

### `SERVICE_DAILY_FACT_TABLE`

In [ ]:
# query = """
# ADD FOREIGN KEY (time_id) REFERENCES "TIME_DIMENSION"(time_id),
# ADD FOREIGN KEY (customer_id) REFERENCES "CUSTOMER_DIMENSION"(customer_id),
# ADD FOREIGN KEY (courier_id) REFERENCES "COURIER_DIMENSION"(courier_id),
# ADD FOREIGN KEY (office_id) REFERENCES "OFFICE_DIMENSION"(office_id);
# """

# alter_table("SERVICE_DAILY_FACT_TABLE", query)

IntegrityError: (psycopg2.errors.ForeignKeyViolation) insert or update on table "SERVICE_DAILY_FACT_TABLE" violates foreign key constraint "SERVICE_DAILY_FACT_TABLE_courier_id_fkey"
DETAIL:  Key (courier_id)=(83) is not present in table "COURIER_DIMENSION".

[SQL: 
			ALTER TABLE "SERVICE_DAILY_FACT_TABLE"
			
ADD FOREIGN KEY (time_id) REFERENCES "TIME_DIMENSION"(time_id),
ADD FOREIGN KEY (courier_id) REFERENCES "COURIER_DIMENSION"(courier_id),
ADD FOREIGN KEY (office_id) REFERENCES "OFFICE_DIMENSION"(office_id);

			]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

### `SERVICE_HOUR_FACT_TABLE`

In [ ]:
# query = """
# ADD FOREIGN KEY (time_id) REFERENCES "TIME_DIMENSION"(time_id),
# ADD FOREIGN KEY (customer_id) REFERENCES "CUSTOMER_DIMENSION"(customer_id),
# ADD FOREIGN KEY (courier_id) REFERENCES "COURIER_DIMENSION"(courier_id),
# ADD FOREIGN KEY (office_id) REFERENCES "OFFICE_DIMENSION"(office_id);
# """

# alter_table("SERVICE_HOUR_FACT_TABLE", query)

### `UPDATES_FACT_TABLE`

In [ ]:
query = """
ADD FOREIGN KEY (update_id) REFERENCES "COURIER_DIMENSION"(courier_id),
ADD FOREIGN KEY (time_id) REFERENCES "TIME_DIMENSION"(time_id);
"""

alter_table("UPDATES_FACT_TABLE", query)